In [75]:
import tkinter as tk
from tkinter import ttk
import tkinter.font as font
from tkinter import messagebox
import pandas as pd
from datetime import datetime 
import os
from openpyxl import load_workbook

def abrir_janela_operacoes(nome, conta,cpf):
    janela_operacoes = tk.Tk()
    janela_operacoes.title(f"Bem vindo {str(nome).upper()}")
    janela_operacoes.geometry("400x500")
    janela_operacoes.configure(background='#ffffff')

    largura = janela_operacoes.winfo_screenwidth()
    altura = janela_operacoes.winfo_screenheight()

    pos_x = (largura // 2 - 400 // 2)
    pos_y = (altura // 2 - 500 // 2)

    janela_operacoes.geometry(f"400x500+{pos_x}+{pos_y}")

    label_nome = tk.Label(janela_operacoes,
                            text=f'Nome: {nome}',
                            font=custom_font,
                            bg='#ffffff')
    label_nome.pack(pady=10)

    label_conta = tk.Label(janela_operacoes,
                            text=f'Número da Conta: {conta}',
                            font=custom_font,
                            bg='#ffffff')
    label_conta.pack(pady=10)

    saldo = 0.0
    saldo_label = tk.Label(janela_operacoes,
                            text=f'Saldo: R$ {saldo:.2f}',
                            font=custom_font,
                            bg='#ffffff')
    saldo_label.pack(pady=10)

    entry = tk.Entry(janela_operacoes,
                        font="Arial 30",
                        bg='#ffffff')
    entry.pack(pady=10,padx=20)

    def sair():
        janela_operacoes.destroy()

    def calcular_saldo(conta):
        
        arquivo_excel = "Base_Dados.xlsx"

        if os.path.exists(arquivo_excel):
            try:
                df_transacoes = pd.read_excel(arquivo_excel,sheet_name="Transações")
                transacoes_deposito = df_transacoes.loc[df_transacoes['Conta'] == int(conta)].loc[df_transacoes['Tipo'] == str("Depósito"),'Valor'].sum()
                transacoes_saque = df_transacoes.loc[df_transacoes['Conta'] == int(conta)].loc[df_transacoes['Tipo'] == str("Saque"),'Valor'].sum()     
                saldo = transacoes_deposito - transacoes_saque
                
            except Exception as e:
                messagebox.showerror("Erro",f"Erro ao ler a planilha: {e}")
            
        else:
            print("Arquivo não encontrado")

        return saldo
    
    saldo = calcular_saldo(conta)   
    saldo_label.config(text="Saldo: R$ {:.2f}".format(saldo))

    def depositar():
        nonlocal saldo
        valor = float(entry.get().strip()) #end-1c ultimo carater do texto
        saldo += valor
        entry.delete(0,"end")
        saldo_label['text'] = f'Saldo: R$ {saldo:.2f}'

        data_atual = datetime.now().strftime('%d/%m/%Y')

        dados_transacao = {
            'Conta':int(conta),
            'Nome':str(nome),
            'CPF': cpf,
            'Tipo':"Depósito",
            'Valor':valor,
            'Data':data_atual
        }

        arquivo_excel = "Base_Dados.xlsx"

        if os.path.exists(arquivo_excel):
            workbook = load_workbook(arquivo_excel)

            if 'Transações' in workbook.sheetnames:
                sheet = workbook['Transações']
            else:
                sheet = workbook.create_sheet('Transações')
        else:
            print("Arquivo não encontrado")

        nova_linha = [dados_transacao['Conta'],dados_transacao['Nome'],
                    dados_transacao['CPF'],dados_transacao['Tipo'],
                    dados_transacao['Valor'],dados_transacao['Data']]

        sheet.append(nova_linha)
        workbook.save(arquivo_excel)

        messagebox.showinfo("Déposito","Déposito realizado com sucesso")

    def sacar():
        nonlocal saldo
        valor = float(entry.get().strip()) #end-1c ultimo carater do texto
        
        if valor <= 2000 and valor <= saldo:            
            saldo -= valor
            entry.delete(0,"end")
            saldo_label['text'] = f'Saldo: R$ {saldo:.2f}'

            data_atual = datetime.now().strftime('%d/%m/%Y')

            dados_transacao = {
                'Conta':int(conta),
                'Nome':str(nome),
                'CPF': cpf,
                'Tipo':"Saque",
                'Valor':valor,
                'Data':data_atual
            }

            arquivo_excel = "Base_Dados.xlsx"

            if os.path.exists(arquivo_excel):
                workbook = load_workbook(arquivo_excel)

                if 'Transações' in workbook.sheetnames:
                    sheet = workbook['Transações']
                else:
                    sheet = workbook.create_sheet('Transações')
            else:
                print("Arquivo não encontrado")

            nova_linha = [dados_transacao['Conta'],dados_transacao['Nome'],
                        dados_transacao['CPF'],dados_transacao['Tipo'],
                        dados_transacao['Valor'],dados_transacao['Data']]

            sheet.append(nova_linha)
            workbook.save(arquivo_excel)

            messagebox.showinfo("Saque","Saque realizado com sucesso")

        elif valor > 2000:
            messagebox.showwarning("Limite Excedido","O valor máximo do saque é de R$ 2.000,00")
        
        else:            
            messagebox.showwarning("Saldo Insuficiente","Saldo Insuficiente para realizar o saque")

    def ver_transacoes():
        transacoes_windows = tk.Toplevel(janela_operacoes)
        transacoes_windows.title("Transações")
        transacoes_windows.configure(bg='#ffffff')

        largura = transacoes_windows.winfo_screenwidth()
        altura = transacoes_windows.winfo_screenheight()

        pos_x = (largura // 2 - 1250 // 2)
        pos_y = (altura // 2 - 300 // 2)

        transacoes_windows.geometry(f"1250x300+{pos_x}+{pos_y}")


        filtro_frame = tk.Frame(transacoes_windows,bg='#ffffff')
        filtro_frame.pack()

        data_inicio_label = tk.Label(filtro_frame,
                                    text="Data de Início: ",
                                    font="Arial 18",bg='#ffffff')
        data_inicio_label.pack(side=tk.LEFT)
        dat_inicio_entry = tk.Entry(filtro_frame,
                                    font="Arial 18",bg='#ffffff')
        dat_inicio_entry.pack(side=tk.LEFT)

        data_fim_label = tk.Label(filtro_frame,
                                    text="Data de Fim: ",
                                    font="Arial 18",bg='#ffffff')
        data_fim_label.pack(side=tk.LEFT)
        dat_fim_entry = tk.Entry(filtro_frame,
                                    font="Arial 18",bg='#ffffff')
        dat_fim_entry.pack(side=tk.LEFT)

        tipo_label = tk.Label(filtro_frame,
                                    text="Tipo:",
                                    font="Arial 18",bg='#ffffff')
        tipo_label.pack(side=tk.LEFT)
        tipo_entry = tk.Entry(filtro_frame,
                                    font="Arial 18",bg='#ffffff')
        tipo_entry.pack(side=tk.LEFT)

        filtrar_button = tk.Button(filtro_frame,
                                    font="Arial 18",
                                    text="Filtrar",bg='#ffffff',
                                    command=lambda:aplicar_filtro())
        filtrar_button.pack(side=tk.LEFT)
        
        saldo_label = tk.Label(transacoes_windows,
                                    text="Saldo: R$ {:.2f}".format(saldo),
                                    font="Arial 18",bg='#ffffff')
        saldo_label.pack()
        
        treeview = ttk.Treeview(transacoes_windows,
                                columns=('Conta','Nome','CPF','Tipo','Valor','Data'),
                                show='headings')
        treeview.pack()

        treeview.heading('Conta',text='Conta')
        treeview.heading('Nome',text='Nome')
        treeview.heading('CPF',text='CPF')
        treeview.heading('Tipo',text='Tipo')
        treeview.heading('Valor',text='Valor')
        treeview.heading('Data',text='Data')

        style = ttk.Style()
        style.configure('Treeview',font=("Arial 16"))

        arquivo_excel = "Base_Dados.xlsx"

        if os.path.exists(arquivo_excel):
            workbook = load_workbook(arquivo_excel)

            if 'Transações' in workbook.sheetnames:
                sheet = workbook['Transações']
                
                def aplicar_filtro():

                    treeview.delete(*treeview.get_children())

                    data_inicio = dat_inicio_entry.get()
                    data_fim = dat_fim_entry.get()
                    tipo = tipo_entry.get()

                    for row in sheet.iter_rows(min_row=2,values_only=True):
                        if row[0] == int(conta):
                            data_transacao = datetime.strptime(row[5],"%d/%m/%Y")

                            if data_inicio and data_transacao < datetime.strptime(data_inicio,"%d/%m/%Y"):
                                continue
                            if data_fim and data_transacao > datetime.strptime(data_fim,"%d/%m/%Y"):
                                continue
                            if tipo and row[3] != tipo:
                                continue

                            row = list(row)

                            row[5] = data_transacao.strftime("%d/%m/%Y")

                            row = tuple(row)
                            treeview.insert('','end',values=row)
                
            aplicar_filtro()

    depositar_button = tk.Button(janela_operacoes,
                            text=f'Depósitar',
                            font="Arial, 20",
                            bg='#ffffff',
                            width=20,
                            command=depositar)
    depositar_button.pack(pady=10)

    sacar_button = tk.Button(janela_operacoes,
                            text=f'Sacar',
                            font="Arial, 20",
                            bg='#ffffff',
                            width=20,
                            command=sacar)
    sacar_button.pack(pady=10)

    transacoes_button = tk.Button(janela_operacoes,
                            text=f'Transações',
                            font="Arial, 20",
                            bg='#ffffff',
                            width=20,command=ver_transacoes)
    transacoes_button.pack(pady=10)
    
    sair_button = tk.Button(janela_operacoes,
                            text=f'Sair',
                            font="Arial, 20",
                            bg='#ffffff',
                            width=20,
                            command=sair)
    sair_button.pack(pady=10)

    janela_operacoes.mainloop()

def verificar_login(entry_conta,entry_senha):
    conta = entry_conta.get()
    senha = entry_senha.get()

    entry_conta.delete(0,tk.END)
    entry_senha.delete(0,tk.END)

    arquivo_excel = "Base_Dados.xlsx"

    try:
        dados= pd.read_excel(arquivo_excel,sheet_name='Usuários')

        for index, row in dados.iterrows():
            if str(row['Conta']) == str(conta) and str(row['Senha']) == str(senha):
                nome = row['Nome']
                cpf = row['CPF']

                janela_login.destroy()

                abrir_janela_operacoes(nome, conta,cpf)

                return
        
        messagebox.showwarning("Não Logado","Conta e/ou Senha incorreta")

    except Exception as e:
        messagebox.showerror("Erro",f"Erro ao ler os dados da planilha {arquivo_excel}: {e}")

janela_login = tk.Tk()
janela_login.title("Caixa Eletrônico")
janela_login.geometry("300x300")
janela_login.configure(background='#ffffff')

largura = janela_login.winfo_screenwidth()
altura = janela_login.winfo_screenheight()

pos_x = (largura // 2 - 300 // 2)
pos_y = (altura // 2 - 300 // 2)

janela_login.geometry(f"300x300+{pos_x}+{pos_y}")

custom_font = font.Font(family="Arial",size=16)

label_conta = tk.Label(janela_login,
                        text="Número da Conta: ",
                        bg='#ffffff',
                        font=custom_font)
label_conta.pack(pady=10)
entry_conta = tk.Entry(janela_login,
                        font=custom_font,
                        bg='#ffffff',)
entry_conta.pack(pady=5)

label_senha = tk.Label(janela_login,
                        text="Senha: ",
                        bg='#ffffff',
                        font=custom_font)
label_senha.pack(pady=10)
entry_senha = tk.Entry(janela_login,
                        font=custom_font,
                        bg='#ffffff',
                        show="*")
entry_senha.pack(pady=5)

button_entrar = tk.Button(janela_login,
text="Entrar",
command=lambda: verificar_login(entry_conta,entry_senha),
font=custom_font,bg='#ffffff')
button_entrar.pack(pady=10)

janela_login.mainloop()